In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split 
from sklearn import metrics
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Data
train_data = pd.read_csv("/kaggle/input/covid19-global-forecasting-week-3/train.csv")
test_df = pd.read_csv("../input/covid19-global-forecasting-week-3/test.csv")
submission = pd.read_csv("../input/covid19-global-forecasting-week-3/submission.csv")

In [ ]:
train_data.info()

In [ ]:
test_df.info()

In [ ]:
import seaborn as sns
plt.figure(figsize = (10,10))
corr = train_data.corr()
sns.heatmap(corr , mask=np.zeros_like(corr, dtype=np.bool) , cmap=sns.diverging_palette(-100,0,as_cmap=True) , square = True)

In [ ]:
df = train_data.fillna('NA').groupby(['Country_Region','Province_State','Date'])['ConfirmedCases'].sum() \
                       .groupby(['Country_Region','Province_State']).max().sort_values() \
                          .groupby(['Country_Region']).sum().sort_values(ascending = False)

top20_countries = pd.DataFrame(df).head(20)
top20_countries

In [ ]:
# Drop Province State as it has no use.
train_data.drop("Province_State", axis=1, inplace=True)

In [ ]:
train_data.head()

In [ ]:
del test_df['Province_State']

In [ ]:
test_df.head()

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
x = X.fillna('NA')

In [ ]:
del train_data['Id']
del test_df['ForecastId']

In [ ]:
X_train = train_data
X_train['Date'] = pd.to_datetime(X_train['Date'], infer_datetime_format=True)
test_df['Date'] = pd.to_datetime(test_df['Date'], infer_datetime_format=True)

In [ ]:
X_train.info()

In [ ]:
X_train_new = X_train.copy()

X_train_new.loc[:, 'Date'] = X_train_new.Date.dt.strftime("%m%d")
X_train_new["Date"]  = X_train_new["Date"].astype(int)

X_train_new.head()

In [ ]:
l = preprocessing.LabelEncoder()

X_train_new.Country_Region = l.fit_transform(X_train_new.Country_Region)

X_train_new.head()

In [ ]:
train_data.head()

In [ ]:
#Preprocessing Test Data

X_Test = test_df.copy()

X_Test.loc[:, 'Date'] = X_Test.Date.dt.strftime("%m%d")
X_Test["Date"]  = X_Test["Date"].astype(int)

X_Test.head()

In [ ]:
X_Test.info()

In [ ]:
X_Test.Country_Region = l.fit_transform(X_Test.Country_Region)

X_Test.head()

In [ ]:
X_train_new.info()

In [ ]:
X = X_train_new.iloc[:,:2] # first 4 coloumns of dataframe with all rows :input values
print(X.shape)
y = X_train_new.iloc[:,2:] #output values
print(y.shape)
X.head()

In [ ]:
cases = y.ConfirmedCases
fatalities = y.Fatalities

In [ ]:
cases.head()

In [ ]:
rf = XGBRegressor(n_estimators = 1250 , random_state = 0)
rf.fit(X,cases)
cases_pred = rf.predict(X_Test)
cases_pred
cases_pred = np.around(cases_pred,decimals = 0)
cases_pred      

In [ ]:
rf = XGBRegressor(n_estimators = 1250 , random_state = 0)
rf.fit(X,fatalities)

In [ ]:
fatalities_pred = rf.predict(X_Test)
fatalities_pred

In [ ]:
fatalities_pred = np.around(fatalities_pred,decimals = 0)
fatalities_pred

In [ ]:
submission['ConfirmedCases'] = cases_pred
submission['Fatalities'] = fatalities_pred

In [ ]:
submission.head()

In [ ]:
submission.to_csv("submission.csv" , index = False)